課題2 (重回帰分析) 
課題１で用いたcsvファイル”salary_1.csv”について再掲する．

* “TIME”列：学位（博士号）取得後の経過年数。
* “PUBS”列：著した論文数． 
* “SALARY”列：年収．
- これについて、”SALARY”の値を従属変数， “TIME”と “PUBS”を説明変数として重回帰分析を行え．変数の標準化をした上で，授業で紹介したステップAICの変数増加法により変数選択を行い，選ばれた変数の組として最も適切なものを以下より選べ．

(i) “TIME”のみ選ばれた
(ii) “PUBS”のみ選ばれた
(iii) “TIME”, “PUBS”ともに選ばれた



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error
import re

In [3]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [4]:
csv_in = 'salary_1.csv'
df = pd.read_csv(csv_in, sep=',')
print(df.shape)
print(df.info())
display(df.head())

(15, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   TIME    15 non-null     int64
 1   PUBS    15 non-null     int64
 2   SALARY  15 non-null     int64
dtypes: int64(3)
memory usage: 488.0 bytes
None


,TIME,PUBS,SALARY
0,3,18,51876
1,6,3,54511
2,3,2,53425
3,8,17,61863
4,9,11,52926


In [5]:
display(df.describe())

,TIME,PUBS,SALARY
count,15.000000,15.000000,15.000000
mean,7.666667,19.933333,53045.600000
std,4.577377,13.822687,7889.768146
min,2.000000,2.000000,39115.000000
25%,5.000000,9.500000,47250.500000
50%,6.000000,18.000000,52926.000000
75%,9.500000,28.500000,60388.500000
max,18.000000,48.000000,66432.000000


In [6]:
X = df.drop(columns='SALARY')  #説明変数, 2D
y = df['SALARY']  #目的変数, 1D
print('X:', X.shape)
display(X.head())
print('y:', y.shape)
print(y.head())

X: (15, 2)


,TIME,PUBS
0,3,18
1,6,3
2,3,2
3,8,17
4,9,11


y: (15,)
0    51876
1    54511
2    53425
3    61863
4    52926
Name: SALARY, dtype: int64


In [7]:
#標準化
X_scaled_ar = scale(X)
y_scaled_ar = scale(y)

In [8]:
X_scaled = pd.DataFrame(X_scaled_ar, columns=X.columns)
y_scaled = pd.Series(y_scaled_ar, name=y.name)
model = sm.OLS(y_scaled, X_scaled)
results_scaled = model.fit()
print(results_scaled.summary())

                                 OLS Regression Results                                
Dep. Variable:                 SALARY   R-squared (uncentered):                   0.530
Model:                            OLS   Adj. R-squared (uncentered):              0.458
Method:                 Least Squares   F-statistic:                              7.345
Date:                Sat, 11 Nov 2023   Prob (F-statistic):                     0.00734
Time:                        15:38:11   Log-Likelihood:                         -15.613
No. Observations:                  15   AIC:                                      35.23
Df Residuals:                      13   BIC:                                      36.64
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

C:\Users\iniad\miniconda3\envs\ds2023\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [9]:
print(results_scaled.params.sort_values(key=np.abs, ascending=False))

TIME    0.570226
PUBS    0.213392
dtype: float64


In [10]:
#変数選択
d_scaled=pd.concat([X_scaled,y_scaled],axis=1)
display(d_scaled.head())

,TIME,PUBS,SALARY
0,-1.055290,-0.144776,-0.153446
1,-0.376889,-1.268036,0.192253
2,-1.055290,-1.342920,0.049775
3,0.075378,-0.219660,1.156799
4,0.301511,-0.668964,-0.015691


In [11]:
#Cell_22.
# forward method for variable selection based on AIC.
# Stepwise feature selection method (forward); 変数増加法による変数選択
def step_aic_forward(model, exog, endog, **kwargs):
    '''
    This function calculates the best subset of explanatory (exogenous) variables based on AIC.
    Both exog and endog can be either str or list.

    Arguments:
        model: model from statsmodels.formula.api
        exog (str or list): explanatory (exogenous) variables
        endog (str or list): objective (endogenous) variables
        kwargs: additional keyword argments for model (data, family, ...)

    Return values:
        model: a model with the smallest AIC
    '''
    
    # Convert exog, endog into 1-d list
    exog = np.r_[[exog]].flatten()
    endog = np.r_[[endog]].flatten()
    remaining = set(exog)
    selected = []  # Selected exogenous variables

    # First, calculate AIC with a constant (no exogs)
    formula_head = 'Q("' + '") + Q("'.join(endog) + '") ~ '
    formula = formula_head + '1'
    aic = model(formula=formula, **kwargs).fit().aic
    print('AIC: {:.3f}, formula: {}'.format(aic, formula))

    current_score, best_new_score = aic, aic

    # Break loop if all exogs are selected or no remaining exogs can improve AIC
    while True:
        score_with_candidates = []
        for candidate in remaining:
            # Calculate AIC for adding an exog one by one
            formula_tail = 'Q("' + '") + Q("'.join(selected + [candidate]) + '")'
            formula = formula_head + formula_tail
            aic = model(formula=formula, **kwargs).fit().aic
            print('AIC: {:.3f}, formula: {}'.format(aic, formula))

            score_with_candidates.append((aic, candidate))

        # Select best_candidate with minimum AIC
        score_with_candidates.sort()
        best_score, best_candidate = score_with_candidates[0]

        # select best_candidate if AIC is improved
        improved = False
        if best_score < current_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_score
            improved = True
            
        if not remaining or not improved: break

    formula = formula_head + 'Q("' + '") + Q("'.join(selected) + '")'
    print('The best formula: {}'.format(formula))
    aic = model(formula=formula, **kwargs).fit().aic
    print('Minimum AIC: {:.3f}'.format(aic))
    
    ret = model(formula, **kwargs).fit()
    ret.model.exog_names_org = [re.sub(r'Q\(\"(.*)\"\)',r'\1',x) for x in list(ret.model.exog_names)]
    ret.model.endog_names_org = re.sub(r'Q\(\"(.*)\"\)',r'\1',ret.model.endog_names)
    return ret

In [12]:
#Cell_23.
#変数選択を実行

header_y = y_scaled.name
header_x = X_scaled.columns
model = step_aic_forward(smf.ols, header_x,header_y, data=d_scaled)

AIC: 44.568, formula: Q("SALARY") ~ 1
AIC: 36.032, formula: Q("SALARY") ~ Q("TIME")
AIC: 40.209, formula: Q("SALARY") ~ Q("PUBS")
AIC: 37.227, formula: Q("SALARY") ~ Q("TIME") + Q("PUBS")
The best formula: Q("SALARY") ~ Q("TIME")
Minimum AIC: 36.032


# The best formula: Q("SALARY") ~ Q("TIME")
